In [180]:
import datasets

dataset = datasets.load_dataset("squad_v2")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [190]:
dataset['train'][0]

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

In [183]:
from transformers import BertTokenizer, BertForQuestionAnswering

tokenizer = BertTokenizer.from_pretrained("csarron/bert-base-uncased-squad-v1")
model = BertForQuestionAnswering.from_pretrained("csarron/bert-base-uncased-squad-v1")

In [193]:
import nltk
nltk.download('punkt')

def find_sentence_from_answer(sentences, answer_start):
    total = 0
    for i, sentence in enumerate(sentences):
        total += len(sentence)
        if total > answer_start:
            return sentences[max(0, i - 1)]
    return sentences[-1]

def convert_to_features(batch):
    answer_start = [answer["answer_start"][0] for answer in batch["answers"]]
    sentences = [nltk.sent_tokenize(context) for context in batch["context"]]
    answer_sentences = [find_sentence_from_answer(sentence, answer_start) for sentence, answer_start in zip(sentences, answer_start)]
    answer_start = [context.find(answer_sentence) for context, answer_sentence in zip(batch["context"], answer_sentences)]
    answer_end = [start + len(answer_sentence) for start, answer_sentence in zip(answer_start, answer_sentences)]
    return {
        'answer_start': answer_start,
        'answer_end': answer_end,
    }
    

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [192]:
dataset = dataset.map(convert_to_features, batched=True, batch_size=8)

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

TypeError: must be str, not NoneType